# ST563 Project Data Retrieval

In [2]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from pandas_datareader.famafrench import get_available_datasets

import warnings
warnings.filterwarnings("ignore")

View the names of the datasets on Kenneth French website.

In [3]:
available_data = get_available_datasets()
available_data

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

Download the 5-factor daily returns and the returns of 6 portfolios constructed on size and value from 2020-2025. Subtract the risk-free rate from the portfolio returns. Replace the spaces in the portfolio names with underscores to avoid errors in future model strings.

In [19]:
start  = '2020-01-01'
end ='2025-01-01'
factors = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily','famafrench', start=start ,end=end)[0]
ports = web.DataReader('6_Portfolios_2x3_daily','famafrench', start=start ,end=end)[0]
ports = ports.sub(factors.RF, axis=0)
ports.columns = ports.columns.str.replace(' ', '_')
print(factors)
print(ports.columns)

            Mkt-RF   SMB   HML   RMW   CMA     RF
Date                                             
2020-01-02    0.86 -0.97 -0.33  0.24 -0.22  0.006
2020-01-03   -0.67  0.30  0.00 -0.13 -0.11  0.006
2020-01-06    0.36 -0.21 -0.55 -0.17 -0.26  0.006
2020-01-07   -0.19 -0.03 -0.25 -0.13 -0.24  0.006
2020-01-08    0.47 -0.16 -0.66 -0.16 -0.18  0.006
...            ...   ...   ...   ...   ...    ...
2024-12-24    1.11 -0.12 -0.05 -0.13 -0.37  0.017
2024-12-26    0.02  1.09 -0.19 -0.44  0.35  0.017
2024-12-27   -1.17 -0.44  0.56  0.41  0.03  0.017
2024-12-30   -1.09  0.24  0.74  0.55  0.14  0.017
2024-12-31   -0.46  0.31  0.71  0.33  0.00  0.017

[1258 rows x 6 columns]
Index(['SMALL_LoBM', 'ME1_BM2', 'SMALL_HiBM', 'BIG_LoBM', 'ME2_BM2',
       'BIG_HiBM'],
      dtype='object')


Download the daily returns of the momentum and short-term reversal factors from 2020-2025. Merge with the 5-factor data and drop the risk-free rate column. Rename a couple of columns to avoid errors in future model strings.

In [20]:
mom = web.DataReader('F-F_Momentum_Factor_daily','famafrench', start=start ,end=end)[0]
st_rev = web.DataReader('F-F_ST_Reversal_Factor_daily','famafrench', start=start ,end=end)[0]
factors = pd.merge(mom, factors, on='Date')
factors = pd.merge(st_rev, factors, on='Date')
factors.drop(['RF'], axis=1, inplace=True)
factors.rename(columns={'Mom   ':'Mom', 'Mkt-RF':'Mkt'}, inplace=True)
print(factors)

            ST_Rev   Mom   Mkt   SMB   HML   RMW   CMA
Date                                                  
2020-01-02   -0.10  0.82  0.86 -0.97 -0.33  0.24 -0.22
2020-01-03    0.26  0.04 -0.67  0.30  0.00 -0.13 -0.11
2020-01-06   -0.23 -0.69  0.36 -0.21 -0.55 -0.17 -0.26
2020-01-07   -0.38  0.01 -0.19 -0.03 -0.25 -0.13 -0.24
2020-01-08    0.31  0.92  0.47 -0.16 -0.66 -0.16 -0.18
...            ...   ...   ...   ...   ...   ...   ...
2024-12-24   -0.32  0.67  1.11 -0.12 -0.05 -0.13 -0.37
2024-12-26   -0.32  0.01  0.02  1.09 -0.19 -0.44  0.35
2024-12-27    0.30 -0.88 -1.17 -0.44  0.56  0.41  0.03
2024-12-30    0.21  0.06 -1.09  0.24  0.74  0.55  0.14
2024-12-31    0.87 -1.07 -0.46  0.31  0.71  0.33  0.00

[1258 rows x 7 columns]


Merge the small-cap, high-value portfolio returns with the updated factors data.

In [21]:
data = pd.merge(ports['SMALL_HiBM'], factors, on='Date')
print(data)

            SMALL_HiBM  ST_Rev   Mom   Mkt   SMB   HML   RMW   CMA
Date                                                              
2020-01-02      -0.276   -0.10  0.82  0.86 -0.97 -0.33  0.24 -0.22
2020-01-03      -0.276    0.26  0.04 -0.67  0.30  0.00 -0.13 -0.11
2020-01-06      -0.016   -0.23 -0.69  0.36 -0.21 -0.55 -0.17 -0.26
2020-01-07      -0.456   -0.38  0.01 -0.19 -0.03 -0.25 -0.13 -0.24
2020-01-08      -0.206    0.31  0.92  0.47 -0.16 -0.66 -0.16 -0.18
...                ...     ...   ...   ...   ...   ...   ...   ...
2024-12-24       0.953   -0.32  0.67  1.11 -0.12 -0.05 -0.13 -0.37
2024-12-26       0.843   -0.32  0.01  0.02  1.09 -0.19 -0.44  0.35
2024-12-27      -1.277    0.30 -0.88 -1.17 -0.44  0.56  0.41  0.03
2024-12-30      -0.247    0.21  0.06 -1.09  0.24  0.74  0.55  0.14
2024-12-31       0.083    0.87 -1.07 -0.46  0.31  0.71  0.33  0.00

[1258 rows x 8 columns]


Create three classes based on the difference in returns of the small-cap, high-value portfolio minus the market. Class 0 is below -0.5, Class 1 is between -0.5 and 0.5, and Class 2 is above 0.5. Stock market returns are noisy so instead of making a class above zero and a class below zero, Class 1 is constructed as near-zero.

In [22]:
data['Comp'] = data['SMALL_HiBM'] - data['Mkt']
data['SMALL_HiBM_Class'] = pd.cut(data['Comp'], 
                               bins=[-np.inf, -.5, .5, np.inf], 
                               labels=[0, 1, 2])                       
print(data)

            SMALL_HiBM  ST_Rev   Mom   Mkt   SMB   HML   RMW   CMA   Comp  \
Date                                                                        
2020-01-02      -0.276   -0.10  0.82  0.86 -0.97 -0.33  0.24 -0.22 -1.136   
2020-01-03      -0.276    0.26  0.04 -0.67  0.30  0.00 -0.13 -0.11  0.394   
2020-01-06      -0.016   -0.23 -0.69  0.36 -0.21 -0.55 -0.17 -0.26 -0.376   
2020-01-07      -0.456   -0.38  0.01 -0.19 -0.03 -0.25 -0.13 -0.24 -0.266   
2020-01-08      -0.206    0.31  0.92  0.47 -0.16 -0.66 -0.16 -0.18 -0.676   
...                ...     ...   ...   ...   ...   ...   ...   ...    ...   
2024-12-24       0.953   -0.32  0.67  1.11 -0.12 -0.05 -0.13 -0.37 -0.157   
2024-12-26       0.843   -0.32  0.01  0.02  1.09 -0.19 -0.44  0.35  0.823   
2024-12-27      -1.277    0.30 -0.88 -1.17 -0.44  0.56  0.41  0.03 -0.107   
2024-12-30      -0.247    0.21  0.06 -1.09  0.24  0.74  0.55  0.14  0.843   
2024-12-31       0.083    0.87 -1.07 -0.46  0.31  0.71  0.33  0.00  0.543   

View the count of the data in each class.

In [29]:
print(len(data[data['SMALL_HiBM_Class']==0]))
print(len(data[data['SMALL_HiBM_Class']==1]))
print(len(data[data['SMALL_HiBM_Class']==2]))

401
504
353


Drop the column made to form the classes and the column with the returns of the portfolio. Print the data one last time to check if it looks correct before saving as a CSV file. 

In [27]:
data.drop(['Comp', 'SMALL_HiBM'], axis=1, inplace=True)
print(data)

            ST_Rev   Mom   Mkt   SMB   HML   RMW   CMA SMALL_HiBM_Class
Date                                                                   
2020-01-02   -0.10  0.82  0.86 -0.97 -0.33  0.24 -0.22                0
2020-01-03    0.26  0.04 -0.67  0.30  0.00 -0.13 -0.11                1
2020-01-06   -0.23 -0.69  0.36 -0.21 -0.55 -0.17 -0.26                1
2020-01-07   -0.38  0.01 -0.19 -0.03 -0.25 -0.13 -0.24                1
2020-01-08    0.31  0.92  0.47 -0.16 -0.66 -0.16 -0.18                0
...            ...   ...   ...   ...   ...   ...   ...              ...
2024-12-24   -0.32  0.67  1.11 -0.12 -0.05 -0.13 -0.37                1
2024-12-26   -0.32  0.01  0.02  1.09 -0.19 -0.44  0.35                2
2024-12-27    0.30 -0.88 -1.17 -0.44  0.56  0.41  0.03                1
2024-12-30    0.21  0.06 -1.09  0.24  0.74  0.55  0.14                2
2024-12-31    0.87 -1.07 -0.46  0.31  0.71  0.33  0.00                2

[1258 rows x 8 columns]


Save the data as a CSV file.

In [28]:
data.to_csv('FamaFrenchCategorical.csv')